# Week 12 — Visualization for ML / NLP
This notebook reproduces the key visualizations for model interpretation:
- Confusion matrix & ROC curve
- Feature importance
- High-dimensional projection (PCA)
- Word patterns and topic-like clusters

It also includes a **2,000‑review IMDB subset** (via Hugging Face)

In [ ]:
# If running locally, you may need:
# %pip install datasets scikit-learn matplotlib seaborn wordcloud bertopic umap-learn
import numpy as np, pandas as pd, matplotlib.pyplot as plt, seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay, RocCurveDisplay
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
plt.rcParams['figure.figsize'] = (7,5)

## Part A — Tabular Demo (Breast Cancer): Confusion Matrix, ROC, Feature Importance

In [ ]:
from sklearn.datasets import load_breast_cancer
bc = load_breast_cancer()
X, y = bc.data, bc.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

rf = RandomForestClassifier(n_estimators=150, random_state=42).fit(X_train, y_train)


ConfusionMatrixDisplay.from_estimator(rf, X_test, y_test, cmap='Blues')

plt.title('Confusion Matrix — Random Forest (Breast Cancer)')
plt.show()

RocCurveDisplay.from_predictions(y_test, rf.predict_proba(X_test)[:,1])
plt.title('ROC Curve — Random Forest (Breast Cancer)')
plt.show()

In [ ]:
imp = pd.Series(rf.feature_importances_, index=bc.feature_names).sort_values().tail(15)
imp.plot(kind='barh')
plt.xlabel('Importance')
plt.title('Top 15 Feature Importances — Random Forest')
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    ConfusionMatrixDisplay, RocCurveDisplay,
    accuracy_score, roc_auc_score
)
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

# ----- data -----
bc = load_breast_cancer()
X, y = bc.data, bc.target
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

# ----- models to compare -----
models = {
    "RandomForest": RandomForestClassifier(n_estimators=150, random_state=42),

    # Naive Bayes (no scaling needed)
    "NaiveBayes": GaussianNB(),

    # Logistic regression + scaling
    "LogReg": make_pipeline(
        StandardScaler(),
        LogisticRegression(max_iter=2000, random_state=42)
    ),

    # SVM with RBF kernel + scaling, enable probability for ROC
    "SVM_RBF": make_pipeline(
        StandardScaler(),
        SVC(kernel="rbf", probability=True, random_state=42)
    ),
}

results = []

# ----- fit, evaluate, collect metrics -----
for name, model in models.items():
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]  # works because we set probability=True where needed

    acc = accuracy_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_proba)
    results.append({"model": name, "accuracy": acc, "roc_auc": auc})

    # confusion matrix per model (optional)
    ConfusionMatrixDisplay.from_estimator(
        model, X_test, y_test, cmap="Blues", normalize="true"
    )
    plt.title(f"Confusion Matrix — {name}")
    plt.tight_layout()
    plt.show()

# ----- compare metrics in a table -----
results_df = pd.DataFrame(results).set_index("model")
print(results_df)

# ----- overlay ROC curves -----
plt.figure(figsize=(7, 6))
for name, model in models.items():
    RocCurveDisplay.from_estimator(model, X_test, y_test, name=name)
plt.plot([0, 1], [0, 1], "k--", label="Random guess")
plt.title("ROC Curves — Model Comparison")
plt.legend()
plt.tight_layout()
plt.show()


## Part B — High‑Dimensional Data: PCA

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Load data
iris = load_iris()
X = iris.data
y = iris.target
feature_names = iris.feature_names
target_names = iris.target_names

# Standardize features before PCA
X_scaled = StandardScaler().fit_transform(X)

# Fit PCA (2 components for visualization)
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

print("Explained variance ratio:", pca.explained_variance_ratio_)


In [ ]:
plt.figure(figsize=(7,5))
colors = ["red", "green", "blue"]
for lab, col in zip([0,1,2], colors):
    plt.scatter(X_pca[y==lab, 0], X_pca[y==lab, 1],
                label=target_names[lab], alpha=0.7, s=60, c=col)

plt.xlabel(f"PC1 ({pca.explained_variance_ratio_[0]*100:.1f}% variance)")
plt.ylabel(f"PC2 ({pca.explained_variance_ratio_[1]*100:.1f}% variance)")
plt.title("Iris — PCA projection (PC1 vs PC2)")
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
# Loadings matrix: shape (n_components, n_features)
loadings = pca.components_.T   # transpose so rows = features

loadings_df = pd.DataFrame(
    loadings,
    columns=["PC1", "PC2"],
    index=feature_names
)

plt.figure(figsize=(6,4))
sns.heatmap(loadings_df, annot=True, cmap="coolwarm", center=0)
plt.title("PCA loadings: original features vs principal components")
plt.xlabel("principal components")
plt.ylabel("original features")
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(7,5))

# scatter of samples
for lab, col in zip([0,1,2], ["red","green","blue"]):
    plt.scatter(X_pca[y==lab, 0], X_pca[y==lab, 1],
                label=target_names[lab], alpha=0.6, s=40, c=col)

# arrows for features (scaled for visibility)
scaling = 2.5
for i, feature in enumerate(feature_names):
    plt.arrow(0, 0,
              pca.components_[0, i]*scaling,
              pca.components_[1, i]*scaling,
              color="black", alpha=0.8, head_width=0.07)
    plt.text(pca.components_[0, i]*scaling*1.1,
             pca.components_[1, i]*scaling*1.1,
             feature, fontsize=9)

plt.xlabel(f"PC1 ({pca.explained_variance_ratio_[0]*100:.1f}% var.)")
plt.ylabel(f"PC2 ({pca.explained_variance_ratio_[1]*100:.1f}% var.)")
plt.title("Iris PCA biplot — samples + feature directions")
plt.axhline(0, color="grey", linewidth=0.5)
plt.axvline(0, color="grey", linewidth=0.5)
plt.legend()
plt.tight_layout()
plt.show()


## Part C — NLP on IMDB (2k subset): Load, Train, Visualize

### 1) Load IMDB (Hugging Face) and take small, balanced subsets

In [ ]:
from datasets import load_dataset
imdb = load_dataset('imdb')
small_train = imdb['train'].shuffle(seed=42).select(range(2000))
small_test  = imdb['test'].shuffle(seed=42).select(range(1000))
train_df = pd.DataFrame(small_train)
test_df  = pd.DataFrame(small_test)
train_df.head()

### 2) Quick EDA: class balance

In [ ]:
labels_map = {0:'Negative', 1:'Positive'}
counts = train_df['label'].value_counts().sort_index()
plt.bar([labels_map[i] for i in counts.index], counts.values)
plt.title('Class Balance in Sampled IMDB (2k)')
plt.xlabel('Class')
plt.ylabel('Count')
plt.show()

### 3) Train TF‑IDF + Logistic Regression and visualize Confusion Matrix / ROC

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

pipe = make_pipeline(TfidfVectorizer(stop_words='english', max_features=5000), LogisticRegression(max_iter=1000, random_state=42))
pipe.fit(train_df['text'], train_df['label'])

ConfusionMatrixDisplay.from_estimator(pipe, test_df['text'], test_df['label'], cmap='Blues')

plt.title('IMDB Sentiment — Confusion Matrix')
plt.show()

RocCurveDisplay.from_estimator(pipe, test_df['text'], test_df['label'])
plt.title('IMDB Sentiment — ROC Curve')
plt.show()

### 4) Top words driving predictions (positive vs negative)

In [ ]:
import numpy as np
vec = pipe.named_steps['tfidfvectorizer']
clf = pipe.named_steps['logisticregression']
feature_names = np.array(vec.get_feature_names_out())
coefs = clf.coef_[0]
idx_pos = np.argsort(coefs)[-15:]
idx_neg = np.argsort(coefs)[:15]

plt.figure(figsize=(7,5))
plt.barh(feature_names[idx_pos], coefs[idx_pos])
plt.title('Top Positive Sentiment Words (coefficients)'); plt.xlabel('Weight'); plt.tight_layout(); plt.show()

plt.figure(figsize=(7,5))
plt.barh(feature_names[idx_neg], -coefs[idx_neg])
plt.title('Top Negative Sentiment Words (absolute coefficients)'); plt.xlabel('Weight'); plt.tight_layout(); plt.show()

### 5) Word clouds

In [ ]:
# %pip install wordcloud  # if needed
from wordcloud import WordCloud
pos_text = ' '.join(train_df[train_df.label==1].text)
neg_text = ' '.join(train_df[train_df.label==0].text)
WordCloud(width=900, height=450, background_color='white').generate(pos_text).to_image()

In [ ]:
WordCloud(width=900, height=450, background_color='white').generate(neg_text).to_image()

In [ ]:
import numpy as np
from PIL import Image
pos_text = ' '.join(train_df[train_df.label==1].text)
neg_text = ' '.join(train_df[train_df.label==0].text)
mask = np.array(Image.open('./GW.png'))

wc = WordCloud(background_color='white', mask = mask, width = mask.shape[1], height = mask.shape[0], max_words = 200).generate(pos_text)

plt.imshow(wc, interpolation = "None")

plt.axis('off')
plt.show()

In [ ]:
import numpy as np
from PIL import Image
pos_text = ' '.join(train_df[train_df.label==1].text)
neg_text = ' '.join(train_df[train_df.label==0].text)
mask = np.array(Image.open('./up.png'))

wc = WordCloud(background_color='white', mask = mask, width = mask.shape[1], height = mask.shape[0], max_words = 200).generate(pos_text)

plt.imshow(wc, interpolation = "None")

plt.axis('off')
plt.show()

In [ ]:
new_review = [["this movie is so bad."], ["wonderful, just wonderful"], ["everything is just average"]]

# Predicted class (0 = negative, 1 = positive)
for review in new_review:
    pred_class = pipe.predict(review)[0]
    print("Predicted class:", pred_class)

# Class probabilities
    proba = pipe.predict_proba(review)[0]
    print("P(negative) =", proba[0])
    print("P(positive) =", proba[1])

### 6) Optional: Topic-like clusters (TF‑IDF + KMeans + 2D projection)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
sample_docs = list(train_df.sample(500, random_state=42).text)

X = TfidfVectorizer(stop_words='english', max_features=5000).fit_transform(sample_docs)
labels = KMeans(n_clusters=2, n_init=10, random_state=42).fit_predict(X)
coords = PCA(n_components=2, random_state=42).fit_transform(X.toarray())

plt.scatter(coords[:,0], coords[:,1], c=labels, s=20, alpha=0.8)
plt.title('Topic-like Clusters — TF-IDF + KMeans (IMDB subset)')
plt.xlabel('Component 1')
plt.ylabel('Component 2')
plt.show()